In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df_tx_bank= pd.read_csv('./transaction_data.csv')
df_tx_bank = df_tx_bank.head(1000).copy()

FileNotFoundError: [Errno 2] No such file or directory: './transaction_data.csv'

In [ ]:
df_processed = df_tx_bank.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
import re

le = LabelEncoder()
df_processed['Transaction Status'] = le.fit_transform(df_processed['Transaction Status'])
df_processed['Transaction ID'] = le.fit_transform(df_processed['Transaction ID'])
df_processed['Transaction Type'] = le.fit_transform(df_processed['Transaction Type'])
df_processed['Sender Account ID'] = le.fit_transform(df_processed['Sender Account ID'])
df_processed['Receiver Account ID'] = le.fit_transform(df_processed['Receiver Account ID'])
df_processed['Device Used'] = le.fit_transform(df_processed['Device Used'])
df_processed['Network Slice ID'] = le.fit_transform(df_processed['Network Slice ID'])
df_processed['Fraud Flag'] = le.fit_transform(df_processed['Fraud Flag'])

df_processed['Timestamp'] = pd.to_datetime(df_processed['Timestamp'])
df_processed['Timestamp'] = df_processed['Timestamp'].dt.hour + df_processed['Timestamp'].dt.minute / 100

In [ ]:
df_reset = df_tx_bank.reset_index(drop=True)

for index, row in df_reset.iterrows():
    nums = re.findall(r'\d+\.?\d*', row['Geolocation (Latitude/Longitude)']) 

    lat = nums[0].replace('.', '')  
    lon = nums[1].replace('.', '')  

    result = int(lat + lon)  
    df_processed.at[index, 'Geolocation (Latitude/Longitude)'] = result
  

In [ ]:
df_processed.drop(columns=['PIN Code'], inplace=True)

df_processed

,Transaction ID,Sender Account ID,Receiver Account ID,Transaction Amount,Transaction Type,Timestamp,Transaction Status,Fraud Flag,Geolocation (Latitude/Longitude),Device Used,Network Slice ID,Latency (ms),Slice Bandwidth (Mbps)
0,949,57,88,495.90,0,10.14,0,1,34052274006,0,2,10,179
1,940,542,266,529.62,2,10.51,1,0,3568951182437,1,1,11,89
2,398,515,918,862.47,2,10.50,0,0,48856623522,1,0,4,53
3,158,409,745,1129.88,1,10.56,1,1,34052274006,1,2,10,127
4,376,567,17,933.24,0,10.25,1,1,557558376173,1,2,20,191
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,716,760,60,1340.76,1,10.49,1,0,3568951182437,1,1,19,224
996,159,954,469,483.36,2,11.00,0,1,557558376173,1,1,12,56
997,82,953,108,199.81,2,10.54,1,0,3405221396917,0,0,14,151
998,187,724,947,1341.86,1,10.59,1,1,5150741396917,1,2,8,95


In [ ]:

scaler = StandardScaler()
X = scaler.fit_transform(df_processed.drop(columns=['Fraud Flag']))
y = columns=df_processed['Fraud Flag'].values


In [ ]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
_, indices = nbrs.kneighbors(X)

def create_edge_index(indices):
    edge_index = []
    for i, neighbors in enumerate(indices):
        for neighbour in neighbors:
            if i != neighbour:
                edge_index.append([i, neighbour])
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()

graph_edge_index = create_edge_index(indices)

In [ ]:
x = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)

data = Data(x=x,edge_index=graph_edge_index,y=y)

In [ ]:
train_mask, test_mask =train_test_split(np.arange(data.num_nodes), test_size=0.2, random_state=42)
data.train_mask = torch.tensor(train_mask, dtype=torch.long)
data.test_mask = torch.tensor(test_mask, dtype=torch.long)

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels,hidden_channels)
        self.conv2 = GCNConv(hidden_channels,out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        x= self.conv2(x, edge_index)

        return x

model = GNN(in_channels=data.num_node_features, hidden_channels=16, out_channels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    y_true = data.y[data.test_mask].cpu().numpy()
    y_pred = pred[data.test_mask].cpu().numpy()

    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return acc,precision,recall,f1

In [ ]:

for epoch in range(1, 100):
    loss = train()
    if epoch % 10 == 0:
        acc,precision,recall,f1 = test()
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')

final_acc, final_precision, final_recall, final_f1 = test()
print(f'Final test accuracy: {final_acc:.4f}, precision: {final_precision:.4f}, recall: {final_recall:.4f}, F1-Score: {final_f1:.4f}')

Epoch: 010, Loss: 0.7056, Test Accuracy: 0.5150, Precision: 0.4521, Recall: 0.3667, F1-Score: 0.4049
Epoch: 020, Loss: 0.6804, Test Accuracy: 0.4850, Precision: 0.4425, Recall: 0.5556, F1-Score: 0.4926
Epoch: 030, Loss: 0.6691, Test Accuracy: 0.5200, Precision: 0.4634, Recall: 0.4222, F1-Score: 0.4419
Epoch: 040, Loss: 0.6600, Test Accuracy: 0.5050, Precision: 0.4579, Recall: 0.5444, F1-Score: 0.4975
Epoch: 050, Loss: 0.6519, Test Accuracy: 0.5000, Precision: 0.4444, Recall: 0.4444, F1-Score: 0.4444
Epoch: 060, Loss: 0.6442, Test Accuracy: 0.5050, Precision: 0.4505, Recall: 0.4556, F1-Score: 0.4530
Epoch: 070, Loss: 0.6374, Test Accuracy: 0.5250, Precision: 0.4725, Recall: 0.4778, F1-Score: 0.4751
Epoch: 080, Loss: 0.6312, Test Accuracy: 0.5050, Precision: 0.4516, Recall: 0.4667, F1-Score: 0.4590
Epoch: 090, Loss: 0.6258, Test Accuracy: 0.5300, Precision: 0.4796, Recall: 0.5222, F1-Score: 0.5000
Final test accuracy: 0.5300, precision: 0.4800, recall: 0.5333, F1-Score: 0.5053
